In [92]:
#import section
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np


In [174]:
class find_school():
    """
    Класс собирает необходимые функции вместе
    """

    def __init__(self, model: str ='sentence-transformers/LaBSE') -> None:
        self.model = SentenceTransformer(model)

    def fit(self, train) -> None:
        """
        !!!в разработке
        """
        self.model.fit()
        
        
    def encode(self, sentence: str|list) -> np.ndarray:
        """ 
        энкодер
        """ 
        return self.model.encode(sentence)

    def find_similar(self, query:np.ndarray, corpus: np.ndarray) -> tuple[int, float]:
        """ 
        Функция возвращает кортеж из номера найденого значения из корпуса и вероятности совпадения
        """
        result = util.semantic_search(query, corpus, top_k=1)
        
        return result[0][0]['corpus_id'], result[0][0]['score']
    
    @staticmethod
    def augment_corpus(self, corpus: list) -> list:
        """
        !!!в разработке
        функция для аугмантации корпуса

        """
        aug_corpus = corpus
        return aug_corpus

    @staticmethod
    def clean_input(self, sentence:str) -> str:  
        """ 
        функция очистки имени школы от лишних символов
        """      
        return sentence.replace(r'[^А-Яа-яёЁA-Za-z№\s0-9]', ' ', regex=True).replace(r'\s+', ' ', regex=True).str.strip()

In [175]:
df = pd.read_csv('reference_schools.csv', index_col=1)
df['merge_name'] = df['name'] + ' ' + df['region']

In [176]:
df.drop(df.columns[0], axis=1, inplace=True)


In [177]:
df.tail()

,name,region,merge_name
school_id,,,
305,Прогресс,Алтайский край,Прогресс Алтайский край
609,"""СШ ""Гвоздика""",Удмуртская республика,"""СШ ""Гвоздика"" Удмуртская республика"
610,"СШОР ""Надежда Губернии",Саратовская область,"СШОР ""Надежда Губернии Саратовская область"
611,КФК «Айсберг»,Пермский край,КФК «Айсберг» Пермский край
1836,"ООО ""Триумф""",Москва,"ООО ""Триумф"" Москва"


In [178]:
query_df = pd.read_csv("Примерное написание.csv", index_col=0)

In [179]:
query_df['clean_name'] = query_df['name'].replace(r'[^А-Яа-яёЁA-Za-z№\s0-9]', ' ', regex=True).replace(r'\s+', ' ', regex=True).str.strip()

In [180]:
query_df['clean_name'].values[10:15]

array(['СШ №2', 'МБУ ЦФКиС г Лобня', 'Рязанская область ХК Рязань ВДВ',
       'ГАУ РО ХК Рязань ВДВ', 'РФСОО ФФККРК'], dtype=object)

In [181]:
model = find_school()

In [182]:
corpus = model.encode(df.merge_name.values)

In [183]:
query_df['np_query']= query_df['clean_name'].apply(model.encode)

In [184]:
query_df[['corpus_idx','score']] = query_df['np_query'].transform(lambda x: model.find_similar(x, corpus)).apply(pd.Series)

In [185]:
query_df

,name,clean_name,np_query,corpus_idx,score
school_id,,,,,
1836,"ООО ""Триумф""",ООО Триумф,"[-0.0020779932, -0.023952289, -0.02229752, 0.0...",287.0,0.731663
1836,"Москва, СК ""Триумф""",Москва СК Триумф,"[0.000107351974, -0.035497945, -0.020385103, -...",287.0,0.867980
610,"СШОР ""Надежда Губернии",СШОР Надежда Губернии,"[-0.043393172, -0.0060612764, 0.01834402, 0.01...",285.0,0.720434
610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...",Саратовская область ГБУСО СШОР Надежда Губернии,"[0.020036744, -0.05076504, 0.02466188, 0.03609...",285.0,0.913695
609,"""СШ ""Гвоздика""",СШ Гвоздика,"[-0.033360552, -0.049458038, -0.023155967, -0....",284.0,0.624845
...,...,...,...,...,...
3,"Республика Татарстан, СШОР ФСО Авиатор",Республика Татарстан СШОР ФСО Авиатор,"[-0.0022777987, -0.07397329, -0.005642348, 0.0...",136.0,0.804658
3,"СШОР ФСО Авиатор, Республика Татарстан",СШОР ФСО Авиатор Республика Татарстан,"[-0.00314746, -0.06956039, -0.014659062, 0.033...",136.0,0.774821
3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",Республика Татарстан МБУ ДО СШОР ФСО Авиатор,"[0.01674769, -0.059445653, 0.0025185728, 0.035...",136.0,0.773810


In [199]:
#добавляем столбец с индексами найденых школ путем сопоставления индекса корпуса и индекса датафрема с эталонами
query_df['school_id'] = [df.index[int(x)] for x in query_df.corpus_idx]

In [200]:
sum(query_df['school_id'] == query_df.index)/query_df.shape[0]

0.6793296089385474